In [1]:
from tensorflow.keras.layers import Softmax, Multiply
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Reshape

In [2]:
df_ratings = pd.read_csv('data/preprocessed/ratings.csv')
df_personality = pd.read_csv('data/preprocessed/personality.csv')
df = df_ratings.merge(df_personality, on='user_id')

In [3]:
# Find the number of unique users and movies in the dataset
df_ratings = pd.read_csv('data/preprocessed/ratings.csv')
number_of_unique_users = df_ratings.user_id.unique().shape[0]
number_of_unique_movies = df_ratings.movie_id.unique().shape[0]

number_of_personality_dims = 5

In [4]:
# Define the embedding dimensions
embedding_dim = 16

# Define the input layers
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))
personality_input = Input(shape=(number_of_personality_dims,))

# Create user and movie embeddings
user_embedding = Embedding(number_of_unique_users, embedding_dim)(user_input)
user_embedding = Flatten()(user_embedding)

movie_embedding = Embedding(number_of_unique_movies, embedding_dim)(movie_input)
movie_embedding = Flatten()(movie_embedding)

# Apply Softmax to personality scores
personality_scores = Softmax()(personality_input)

# Flatten personality scores
personality_flat = Flatten()(personality_scores)

# Concatenate user, movie, and personality embeddings
concat = Concatenate()([user_embedding, movie_embedding, personality_flat])

# Create a 4-layer MLP
layer1 = Dense(64, activation='relu')(concat)
layer2 = Dense(32, activation='relu')(layer1)
layer3 = Dense(16, activation='relu')(layer2)
output = Dense(1)(layer3)  # Output layer for rating prediction

# Create the model
model = Model(inputs=[user_input, movie_input, personality_input], outputs=output)

In [5]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [6]:
# Train the model for 10 epochs with a batch size of 64
df_train = pd.read_csv('data/preprocessed/ratings-train.csv')

train_user_ids = df_train['user_id'].values
train_movie_ids = df_train['movie_id'].values
train_ratings = df_train['rating'].values
train_personality = df_train[['openness', 'agreeableness', 'neuroticism', 'conscientiousness', 'extraversion']].values
train_personality_input = np.array(train_personality)  # Keep all 5 personality scores

model.fit([train_user_ids, train_movie_ids, train_personality_input], train_ratings, epochs=10, batch_size=64)

Epoch 1/10
13375/13375 [==============================] - 121s 9ms/step - loss: 0.7002
Epoch 2/10
13375/13375 [==============================] - 121s 9ms/step - loss: 0.5969
Epoch 3/10
13375/13375 [==============================] - 114s 9ms/step - loss: 0.5675
Epoch 4/10
13375/13375 [==============================] - 102s 8ms/step - loss: 0.5425
Epoch 5/10
13375/13375 [==============================] - 175s 13ms/step - loss: 0.5203
Epoch 6/10
13375/13375 [==============================] - 192s 14ms/step - loss: 0.5007
Epoch 7/10
13375/13375 [==============================] - 189s 14ms/step - loss: 0.4839
Epoch 8/10
13375/13375 [==============================] - 182s 14ms/step - loss: 0.4692
Epoch 9/10
13375/13375 [==============================] - 180s 13ms/step - loss: 0.4561
Epoch 10/10
13375/13375 [==============================] - 178s 13ms/step - loss: 0.4451


In [ ]:
model.save('ncf-soft-labeled-personality.h5')